In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
list=[['2019-04-26 01:19:10','2019-04-26 01:19:35']]

df=spark.createDataFrame(list,['prev_time','CAL_COMPLETION_TIME'])

df1=df.withColumn("CAL_COMPLETION_TIME", F.to_timestamp("CAL_COMPLETION_TIME"))\
  .withColumn("prev_time", F.to_timestamp("prev_time"))
df1.createOrReplaceTempView("df1")
df1.show()

+-------------------+-------------------+
 prev_time|CAL_COMPLETION_TIME|
+-------------------+-------------------+
2019-04-26 01:19:10|2019-04-26 01:19:35|
+-------------------+-------------------+

In [3]:
df1.withColumn("sub", F.when(((F.col("CAL_COMPLETION_TIME").cast("long")-F.col("prev_time").cast("long"))/60 < 30), F.lit("LESSTHAN30"))\
               .otherwise(F.lit("GREATERTHAN"))).show()

+-------------------+-------------------+----------+
 prev_time|CAL_COMPLETION_TIME| sub|
+-------------------+-------------------+----------+
2019-04-26 01:19:10|2019-04-26 01:19:35|LESSTHAN30|
+-------------------+-------------------+----------+

In [4]:
df1.createOrReplaceTempView("df1")
spark.sql("select prev_time, CAL_COMPLETION_TIME, IF(((CAST(CAL_COMPLETION_TIME as bigint) - CAST(prev_time as bigint))/60)<30,'LESSTHAN30','GREATER') as difference_duration from df1").show()

+-------------------+-------------------+-------------------+
 prev_time|CAL_COMPLETION_TIME|difference_duration|
+-------------------+-------------------+-------------------+
2019-04-26 01:19:10|2019-04-26 01:19:35| LESSTHAN30|
+-------------------+-------------------+-------------------+

In [5]:
case 
    when t1.prev_time <> t1.prev_time_calc and t1."CAL_COMPLETION_TIME" - t1.prev_time < interval '30 min' 
      then t1.next_time_calc - t1.prev_time_calc
    when (t1.next_time <> t1.next_time_calc and t1.next_time - t1."CAL_COMPLETION_TIME" < interval '30 min') or (t1.next_time - t1."CAL_COMPLETION_TIME" < interval '30 min')
      then t1.next_time_calc - t1."CAL_COMPLETION_TIME"
  else null
  end min_diff

In [6]:
list=[[1,1,5],
       [1,2,7],
        [1,3,6],
        [1,4,9],
        [1,5,4],
        [1,6,3],
        [2,1,9],
        [2,2,10],
        [2,3,8],
        [2,4,12],
         [2,5,4]]

df=spark.createDataFrame(list,['id','rowID','value'])
df.show()

+---+-----+-----+
 id|rowID|value|
+---+-----+-----+
 1| 1| 5|
 1| 2| 7|
 1| 3| 6|
 1| 4| 9|
 1| 5| 4|
 1| 6| 3|
 2| 1| 9|
 2| 2| 10|
 2| 3| 8|
 2| 4| 12|
 2| 5| 4|
+---+-----+-----+

In [7]:
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType
@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
def grouped_map(df1):
   for i in range(1, len(df1)):
        if df1.loc[i, 'value']>df1.loc[i-1,'value']:
           df1.loc[i,'value']=df1.loc[i-1,'value']

   return df1
df.groupby("id").apply(grouped_map).show()

+---+-----+-----+
 id|rowID|value|
+---+-----+-----+
 1| 1| 5|
 1| 2| 5|
 1| 3| 5|
 1| 4| 5|
 1| 5| 4|
 1| 6| 3|
 2| 1| 9|
 2| 2| 9|
 2| 3| 8|
 2| 4| 8|
 2| 5| 4|
+---+-----+-----+

In [8]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
def grouped_map(df1):
   for i in range(1, len(df1)):
        if df1.loc[i, 'value']>df1.loc[i-1,'value']:
           df1.loc[i,'value']=df1.loc[i-1,'value']
            
   return df1
df.groupby().apply(grouped_map).show() 

+-----+-----+
rowID|value|
+-----+-----+
 1| 5|
 2| 5|
 3| 5|
 4| 5|
 5| 4|
 6| 3|
+-----+-----+

In [9]:
def scalar(df1):
   for i in range(1, len(df1)):
        if df1.loc[i, 'value']>df1.loc[i-1,'value']:
           df1.loc[i,'value']=df1.loc[i-1,'value']

df1

,rowID,value
0,1,5
1,2,5
2,3,5
3,4,5
4,5,4
5,6,3


In [10]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
@pandas_udf("rowID integer, value double", PandasUDFType.GROUPED_MAP)  
def normalize(pdf):
    import pandas as pd
    value = pdf.value
    return pdf.assign(value=value.mean())
df.groupby().apply(normalize).show() 

+-----+-----------------+
rowID| value|
+-----+-----------------+
 1|5.666666666666667|
 2|5.666666666666667|
 3|5.666666666666667|
 4|5.666666666666667|
 5|5.666666666666667|
 6|5.666666666666667|
+-----+-----------------+

In [11]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
df = spark.createDataFrame(
    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],
    ("id", "v"))  
@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)  
def normalize(pdf):
    v = pdf.v
    return pdf.assign(v=v.mean())
df.groupby().apply(normalize).show() 

+---+---+
 id| v|
+---+---+
 1|4.2|
 1|4.2|
 2|4.2|
 2|4.2|
 2|4.2|
+---+---+

In [12]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
def add_one(a):
    for i in range(1,len(a)):
       if a[i]>a[i-1]:
           a[i]=a[i-1]
    return a
udf1= F.udf(add_one, ArrayType(IntegerType()))
df.agg(F.collect_list("rowID").alias("rowID"),F.collect_list("value").alias("value"))\
  .withColumn("value", udf1("value"))\
  .withColumn("zipped", F.explode(F.struct(("rowID"),("value")))).select("zipped.*").show()



--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o14391.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'explode(named_struct('rowID', `rowID`, 'value', `value`))' due to data type mismatch: input to function explode should be array or map type, not struct<rowID:array<bigint>,value:array<int>>;;
'Project [rowID#7222, value#7230, explode(named_struct(rowID, rowID#7222, value, value#7230)) AS zipped#7233]
+- Project [rowID#7222, add_one(value#7224) AS value#7230]
 +- Aggregate [collect_list(rowID#7143L, 0, 0) AS rowID#7222, collect_list(value#7144L, 0, 0) AS value#7224]
 +- LogicalRDD [rowID#7143L, value#7144L], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
def add_one(a):
    for i in range(1,len(a)):
       if a[i]>a[i-1]:
           a[i]=a[i-1]
    return a
udf1= F.udf(add_one, ArrayType(IntegerType()))
df.agg(F.collect_list("rowID").alias("rowID"),F.collect_list("value").alias("value"))\
  .withColumn("value", udf1("value"))\
  .withColumn("zipped", F.explode(F.arrays_zip("rowID","value"))).select("zipped.*").show()

+-----+-----+
rowID|value|
+-----+-----+
 1| 5|
 2| 5|
 3| 5|
 4| 5|
 5| 4|
 6| 3|
+-----+-----+

In [14]:
a=[5,7,6,9,4,3]
for i in range(1,len(a)):
  if a[i]>a[i-1]:
    a[i]=a[i-1]

    
a

Out[209]: [5, 5, 5, 5, 4, 3]

In [15]:
for i in range(1,len(a)):
  print(a[i-1])

5
7
6
9
4

In [16]:
w=Window().partitionBy().orderBy("rowID")
df.withColumn("lag", F.lag("value").over(w))\
  .withColumn("lag2", F.when((F.col("value")>F.col("lag")),F.lit(1)).otherwise(F.lit(0))).show()

+-----+-----+----+----+
rowID|value| lag|lag2|
+-----+-----+----+----+
 1| 5|null| 0|
 2| 7| 5| 1|
 3| 6| 7| 0|
 4| 9| 6| 1|
 5| 4| 9| 0|
 6| 3| 4| 0|
+-----+-----+----+----+

In [17]:
import pandas as pd
import datetime

data = {'date': ['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05', '2014-01-06', '2014-01-07'],
     'id': [1, 2, 2, 3, 4, 4, 5], 'name': ['Darren', 'Sabrina', 'Steve', 'Sean', 'Ray', 'Stef', 'Dany']}
data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'])
df=spark.createDataFrame(data)
df.show()

+-------------------+---+-------+
 date| id| name|
+-------------------+---+-------+
2014-01-01 00:00:00| 1| Darren|
2014-01-02 00:00:00| 2|Sabrina|
2014-01-03 00:00:00| 2| Steve|
2014-01-04 00:00:00| 3| Sean|
2014-01-05 00:00:00| 4| Ray|
2014-01-06 00:00:00| 4| Stef|
2014-01-07 00:00:00| 5| Dany|
+-------------------+---+-------+

In [18]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df=spark.createDataFrame(data)

w=Window().partitionBy("id").orderBy((F.col("date")).cast("long")).rangeBetween(-(86400*2),Window.currentRow)
df.withColumn("no_distinct", F.size(F.array_distinct(F.collect_list("name").over(w))))\
  .withColumn("no_distinct", F.when(F.col("no_distinct")>1, F.lit(1)).otherwise(F.lit(0)))\
  .orderBy(F.col("date")).show()



+-------------------+---+-------+-----------+
 date| id| name|no_distinct|
+-------------------+---+-------+-----------+
2014-01-01 00:00:00| 1| Darren| 0|
2014-01-02 00:00:00| 2|Sabrina| 0|
2014-01-03 00:00:00| 2| Steve| 1|
2014-01-04 00:00:00| 3| Sean| 0|
2014-01-05 00:00:00| 4| Ray| 0|
2014-01-06 00:00:00| 4| Stef| 1|
2014-01-07 00:00:00| 5| Dany| 0|
+-------------------+---+-------+-----------+

In [19]:
for i in df1.collect()[3]:
  print(i)

2014-01-04 00:00:00
3
Sean
0

In [20]:
df1.collect()

Out[179]: [Row(date=datetime.datetime(2014, 1, 1, 0, 0), id=1, name='Darren', no_distinct=0),
 Row(date=datetime.datetime(2014, 1, 2, 0, 0), id=2, name='Sabrina', no_distinct=0),
 Row(date=datetime.datetime(2014, 1, 3, 0, 0), id=2, name='Steve', no_distinct=1),
 Row(date=datetime.datetime(2014, 1, 4, 0, 0), id=3, name='Sean', no_distinct=0),
 Row(date=datetime.datetime(2014, 1, 5, 0, 0), id=4, name='Ray', no_distinct=0),
 Row(date=datetime.datetime(2014, 1, 6, 0, 0), id=4, name='Stef', no_distinct=1),
 Row(date=datetime.datetime(2014, 1, 7, 0, 0), id=5, name='Dany', no_distinct=0)]

In [21]:
df1.dtypes

Out[181]: [('date', 'timestamp'),
 ('id', 'bigint'),
 ('name', 'string'),
 ('no_distinct', 'int')]

In [22]:
list=[['profile','CREATE_CARD','2020-02-24 03:07:04']]
df=spark.createDataFrame(list,['attribute','operation','timestamp'])
df.show()

+---------+-----------+-------------------+
attribute| operation| timestamp|
+---------+-----------+-------------------+
 profile|CREATE_CARD|2020-02-24 03:07:04|
+---------+-----------+-------------------+

In [23]:
df1=df.withColumn("params", F.array(F.struct(F.lit('kobo1'),F.lit('kobo2'),F.lit('kobo3'))))
df2=df.withColumn("params", F.array(F.struct(F.lit('kobo1'),F.lit('kobo2'),F.lit('kobo3'),F.lit('kobo4'),F.lit('kobo5'))))

In [24]:
import re
string=df1.dtypes[3][1]
d=re.findall('\d+', string )
a=','.join(['x.'+'col'+x for x in d])


In [25]:
import re
string=df2.dtypes[3][1]
d=re.findall('\d+', string )
a=','.join(['x.'+'col'+x for x in d])

In [26]:
import re
string=df1.dtypes[3][1]
d=re.findall('\d+', string )
a=','.join(['x.'+'col'+x for x in d])
a='x.col1,x.col2,x.col3'
df1.withColumn("params", F.flatten(F.expr("transform(params, x -> array('"+ a +"'))"))).printSchema()


root
-- attribute: string (nullable = true)
-- operation: string (nullable = true)
-- timestamp: string (nullable = true)
-- params: array (nullable = false)
 |-- element: string (containsNull = false)

In [27]:
df1=df.withColumn("params", F.struct(F.lit('kobo1'),F.lit('kobo2'),F.lit('kobo3')))
df2=df.withColumn("params", F.struct(F.lit('kobo1'),F.lit('kobo2'),F.lit('kobo3'),F.lit('kobo4'),F.lit('kobo5')))

In [28]:
df1=df1.withColumn("params", F.array(*(['params.*'])))
df2=df2.withColumn("params", F.array(*(['params.*'])))


df1.union(df2).show(truncate=False)



+---------+-----------+-------------------+-----------------------------------+
attribute|operation |timestamp |params |
+---------+-----------+-------------------+-----------------------------------+
profile |CREATE_CARD|2020-02-24 03:07:04|[kobo1, kobo2, kobo3] |
profile |CREATE_CARD|2020-02-24 03:07:04|[kobo1, kobo2, kobo3, kobo4, kobo5]|
+---------+-----------+-------------------+-----------------------------------+

In [29]:
df1.withColumn("params", F.flatten("params")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o969.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'flatten(`params`)' due to data type mismatch: The argument should be an array of arrays, but '`params`' is of struct<col1:string,col2:string,col3:string> type.;;
'Project [attribute#2, operation#3, timestamp#4, flatten(params#52) AS params#94]
+- Project [attribute#2, operation#3, timestamp#4, named_struct(col1, kobo1, col2, kobo2, col3, kobo3) AS params#52]
 +- LogicalRDD [attribute#2, operation#3, timestamp#4], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressions

In [30]:
df1.withColumn("params", F.expr("""transform(array(params),x-> x)""")).show()

+---------+-----------+-------------------+--------------------+
attribute| operation| timestamp| params|
+---------+-----------+-------------------+--------------------+
 profile|CREATE_CARD|2020-02-24 03:07:04|[[kobo1, kobo2, k...|
+---------+-----------+-------------------+--------------------+

In [31]:
list=[['11/9/18 14:11'],
      ['11/9/18 14:27'],
      ['11/9/18 14:42'],
      ['11/9/18 14:57']]

df=spark.createDataFrame(list,['timestamp'])
df.show()

+-------------+
 timestamp|
+-------------+
11/9/18 14:11|
11/9/18 14:27|
11/9/18 14:42|
11/9/18 14:57|
+-------------+

In [32]:
df.withColumn("timestamp", F.to_timestamp("timestamp","dd/MM/yy HH:mm"))\
  .withColumn("time",F.date_format("timestamp","HH:mm"))\
  .withColumn("current_date_formated",F.date_format("timestamp", "M/d/yy")).show()

+-------------------+-----+---------------------+
 timestamp| time|current_date_formated|
+-------------------+-----+---------------------+
2018-09-11 14:11:00|14:11| 9/11/18|
2018-09-11 14:27:00|14:27| 9/11/18|
2018-09-11 14:42:00|14:42| 9/11/18|
2018-09-11 14:57:00|14:57| 9/11/18|
+-------------------+-----+---------------------+

In [33]:
schema = StructType([StructField("col1", StringType())\
                   ,StructField("col2", IntegerType())\
                   ,StructField("row_number", IntegerType())])

data = [['X', 1, 1], ['Y', 0, 2], ['Z', 2, 3], ['A', 1, 4], ['B', 0, 5], ['C', 0, 6], ['D', 2, 7], ['P', 1, 8], ['Q', 2, 9]]

df = spark.createDataFrame(data,schema=schema)
df.show()

+----+----+----------+
col1|col2|row_number|
+----+----+----------+
 X| 1| 1|
 Y| 0| 2|
 Z| 2| 3|
 A| 1| 4|
 B| 0| 5|
 C| 0| 6|
 D| 2| 7|
 P| 1| 8|
 Q| 2| 9|
+----+----+----------+

In [34]:
w=Window().orderBy("row_number") 
df=df.withColumn("incremental_sum", F.sum("col2").over(w)).withColumn("lag", F.lag("incremental_sum").over(w)).withColumn("incremental_sum", F.when(F.col("incremental_sum")%3==0, F.col("lag")).otherwise(F.col("incremental_sum"))) w2=Window().partitionBy("incremental_sum")
df.withColumn("name", F.concat_ws("", F.collect_list(df.col1).over(w2))).show()

File "<command-2482587633947714>" , line 2 
 df=df.withColumn("incremental_sum", F.sum("col2").over(w)).withColumn("lag", F.lag("incremental_sum").over(w)).withColumn("incremental_sum", F.when(F.col("incremental_sum")%3==0, F.col("lag")).otherwise(F.col("incremental_sum"))) w2=Window().partitionBy("incremental_sum") 
 ^
 SyntaxError : invalid syntax

In [35]:
from pyspark.sql.functions import when
w=Window().orderBy("row_number")
df.withColumn("incremental_sum", F.sum("col2").over(w))\
  .withColumn("lag", F.lag("incremental_sum").over(w))\
  .withColumn("incremental_sum", F.when(F.col("incremental_sum")%3==0, F.col("lag")).otherwise(F.col("incremental_sum")))\
  .groupBy("incremental_sum").agg(F.concat_ws("",F.collect_list("col1")).alias("output_col")).drop("incremental_sum").show()

+----------+
output_col|
+----------+
 XYZ|
 ABCD|
 PQ|
+----------+

In [37]:
from pyspark.sql.functions import when
w=Window().orderBy("row_number")
w1=Window().orderBy(F.col("row_number"))
w2=Window().partitionBy("incremental_sum")
df.withColumn("incremental_sum", F.sum("col2").over(w))\
  .withColumn("incremental_sum", F.when(F.col("incremental_sum")%3==0, F.col("incremental_sum")).otherwise(F.lit(0)))\
  .withColumn("lag", F.lag("incremental_sum").over(w1))\
  .withColumn("incremental_sum", F.when(F.col("incremental_sum")==3, F.lit(0)).when(F.col("lag")==3, F.col("lag")).otherwise(F.col("incremental_sum"))).drop("lag")\
  .withColumn("incremental_sum", F.sum("incremental_sum").over(w))\
  .withColumn("output_col", F.collect_list("col1").over(w2)).show()

+----+----+----------+---------------+----------+
col1|col2|row_number|incremental_sum|output_col|
+----+----+----------+---------------+----------+
 X| 1| 1| 0| [X, Y, Z]|
 Y| 0| 2| 0| [X, Y, Z]|
 Z| 2| 3| 0| [X, Y, Z]|
 A| 1| 4| 3| [A, B, C]|
 B| 0| 5| 3| [A, B, C]|
 C| 0| 6| 3| [A, B, C]|
 D| 2| 7| 9| [D, P]|
 P| 1| 8| 9| [D, P]|
 Q| 2| 9| 18| [Q]|
+----+----+----------+---------------+----------+

In [38]:
from pyspark.sql.functions import when
w=Window().orderBy("row_number")
w2=Window().partitionBy("incremental_sum")
df.withColumn("incremental_sum", F.sum("col2").over(w))\
  .withColumn("lag", F.lag("incremental_sum").over(w))\
  .withColumn("incremental_sum", F.when(F.col("incremental_sum")%3==0, F.col("lag")).otherwise(F.col("incremental_sum")))\
  .withColumn("output_col", F.array_join(F.collect_list("col1").over(w2),''))\
  .select(F.col("output_col")).distinct().show()

+----------+
output_col|
+----------+
 XYZ|
 ABCD|
 PQ|
+----------+

In [39]:
from pyspark.sql.functions import when
w=Window().orderBy("row_number")
df.withColumn("incremental_sum", F.sum("col2").over(w))\
  .withColumn("lag", F.lag("incremental_sum").over(w))\
  .withColumn("incremental_sum", F.when(F.col("incremental_sum")%3==0, F.col("lag")).otherwise(F.col("incremental_sum")))\
  .groupBy("incremental_sum").agg(F.array_join(F.collect_list("col1"),"").alias("output_col")).drop("incremental_sum").show()


+----------+
output_col|
+----------+
 XYZ|
 ABCD|
 PQ|
+----------+

In [40]:
from pyspark.sql import functions as F    
    df1=df_json.select("id","type","name","ppu","topping","batters.*")\
           .withColumn("batter", F.explode("batter"))\
           .select("id","type","name","ppu","topping","batter")
    df1.withColumn("topping", F.explode("topping")).select("id","type","name","ppu","topping.*","batter.*")

In [41]:
list=[[['a', 1.0], [['b', 5.0]]],[[['c', 1.0]], [['d', 2.0]]]]
df=spark.createDataFrame(list,['1','2'])
df.printSchema()

root
-- 1: array (nullable = true)
 |-- element: array (containsNull = true)
 | |-- element: string (containsNull = true)
-- 2: array (nullable = true)
 |-- element: array (containsNull = true)
 | |-- element: string (containsNull = true)

In [42]:
  timestamp
11/9/18 14:11
11/9/18 14:27
11/9/18 14:42
11/9/18 14:57